In [5]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [6]:

def approx_nested_loocv_linear_model_select(X:np.ndarray, y:np.ndarray):
    '''
    Given many features and a target return y
    this function approximates the computation
    of estimation of out of sample sharpe
    There are p competitive models and we need
    to choose one. The estimated sharpe is deplected
    to account for feature selection
    X: n, p array with features
    y: n, vector with target returns    
    '''
    X = np.ascontiguousarray(X)
    y = np.ascontiguousarray(y)
    n, p = X.shape   
    sxy = np.sum(X*y[:,None], axis = 0)
    sx = np.sum(X, axis = 0)
    sy = np.sum(y)
    sxx = np.sum(X*X, axis = 0)
    bi = ((n-1)*(sxy-X*y[:,None]) - (sx-X)*(sy-y)[:,None]) / ((n-1)*(sxx-X*X) - (sx-X)*(sx-X))
    ai = ((sy-y)[:,None] - bi*(sx-X))/(n-1) 
    oos_strategy = y[:,None]*(ai+bi*X)
    # compute best model index
    oos_sharpes = np.mean(oos_strategy, axis = 0) / np.std(oos_strategy, axis = 0)
    best_feature = np.argmax(oos_sharpes)
    
    # approximate nested cv
    # Compute the mean and std excluding the i-th row
    total_sum = np.sum(oos_strategy, axis=0)  # Total sum of all rows
    total_sq_sum = np.sum(oos_strategy**2, axis=0)  # Total squared sum for std calculation
    mean_excl = (total_sum - oos_strategy) / (n - 1)  # Mean excluding each row
    var_excl = (total_sq_sum - oos_strategy**2) / (n - 1) - mean_excl**2  # Variance excluding each row
    std_excl = np.sqrt(var_excl)  # Standard deviation
    # Calculate Sharpe ratios excluding each row
    sr_excl = mean_excl / std_excl
    # Find the index of the maximum Sharpe ratio for each row
    idx_max = np.argmax(sr_excl, axis=1)
    # Get sharpe at indexes
    rn = np.arange(n)
    nested_oos_strategy = oos_strategy[rn, idx_max]     
    nested_oos_sharpe = np.mean(nested_oos_strategy) / np.std(nested_oos_strategy)
    oos_sharpe = oos_sharpes[best_feature]
    return best_feature, nested_oos_sharpe, oos_sharpe


In [35]:

# CHECK THIS IMPL TO CONFIRM IT GIVES THE SAME RESULTS
def approx_nested_loocv_linear_model_select_new(X: np.ndarray, y: np.ndarray, k:int = 5):
    '''
    Approximates out-of-sample Sharpe ratio while penalizing based on sample size,
    including during the nested LOOCV approximation. Handles fully missing features.

    X: n, p array with features (can contain NaNs)
    y: n, vector with target returns (can contain NaNs)
    '''
    n, p = X.shape

    # Mask NaNs in X and y
    X_masked = np.ma.masked_invalid(X)
    y_masked = np.ma.masked_invalid(y)
    valid_mask = (~X_masked.mask) & (~y_masked.mask[:, None])

    # Replace NaNs with 0 for valid computations
    X_filled = X_masked.filled(0)
    y_filled = y_masked.filled(0)

    # Precompute terms for all features
    sxy = np.sum(X_filled * y_filled[:, None] * valid_mask, axis=0)
    sx = np.sum(X_filled * valid_mask, axis=0)
    sy = np.sum(y_filled[:, None] * valid_mask, axis=0)
    sxx = np.sum(X_filled**2 * valid_mask, axis=0)
    n_valid = np.sum(valid_mask, axis=0)

    # Identify features with at least k valid points
    valid_features = n_valid > k
    if not np.any(valid_features):
        raise ValueError("All features are fully missing or invalid.")

    # Filter out invalid features
    sxy = sxy[valid_features]
    sx = sx[valid_features]
    sy = sy[valid_features]
    sxx = sxx[valid_features]
    X_filled = X_filled[:, valid_features]
    valid_mask = valid_mask[:, valid_features]
    n_valid = n_valid[valid_features]

    # Compute slopes (b) and intercepts (a) using LOOCV formulas
    numerator_bi = (n_valid - 1) * (sxy - X_filled * y_filled[:, None]) - (sx - X_filled) * (sy - y_filled[:, None])
    denominator_bi = (n_valid - 1) * (sxx - X_filled**2) - (sx - X_filled)**2
    bi = numerator_bi / denominator_bi
    ai = (sy - y_filled[:, None] - bi * (sx - X_filled)) / (n_valid - 1)

    # Compute out-of-sample strategies
    oos_strategy = y_filled[:, None] * (ai + bi * X_filled) * valid_mask

    # Sharpe ratios for each feature
    mean_oos = np.ma.sum(oos_strategy, axis=0) / n_valid
    std_oos = np.sqrt(np.ma.sum(oos_strategy**2, axis=0) / n_valid - mean_oos**2)
    oos_sharpes = mean_oos / std_oos

    # Penalize Sharpe ratios based on sample size
    max_n_valid = np.max(n_valid)
    sharpe_penalized = oos_sharpes * np.sqrt(n_valid / max_n_valid)

    # Find the best feature among valid ones
    best_feature_index = np.argmax(sharpe_penalized)
    best_feature = np.flatnonzero(valid_features)[best_feature_index]  # Map back to original index

    # --- Nested LOOCV Sharpe with Penalization ---
    total_sum = np.sum(oos_strategy, axis=0)
    total_sq_sum = np.sum(oos_strategy**2, axis=0)

    # Compute leave-one-out mean and variance for each row and feature
    mean_excl = (total_sum - oos_strategy) / (n_valid - 1)
    var_excl = (total_sq_sum - oos_strategy**2) / (n_valid - 1) - mean_excl**2
    std_excl = np.sqrt(var_excl)
    sr_excl = mean_excl / std_excl  # Leave-one-out Sharpe ratios

    # Penalize Sharpe ratios in nested CV
    n_valid_excl = n_valid - 1  # Number of valid points excluding row i
    sr_excl_penalized = sr_excl * np.sqrt(n_valid_excl / max_n_valid)

    # Find the best feature for each row based on penalized Sharpe
    idx_max = np.argmax(sr_excl_penalized, axis=1)
    rn = np.arange(n)
    nested_oos_strategy = oos_strategy[rn, idx_max]

    # get out results without number of samples penalization
    # it is only used for model selection
    nested_oos_sharpe = np.mean(nested_oos_strategy) / np.std(nested_oos_strategy)
    oos_sharpe = oos_sharpes[best_feature_index]

    return best_feature, nested_oos_sharpe, oos_sharpe


np.random.seed(42)

n, p = 10,3

X = np.random.normal(0,1,(n,p))
y = np.random.normal(0,1,n)

X[0,0] = np.nan
X[0,1] = np.nan
X[1,0] = np.nan
X[2,0] = np.nan
X[:,0] *= np.nan

s = time.time()
best_feature, nested_oos_sharpe, oos_sharpe = approx_nested_loocv_linear_model_select_new(X, y)
print('Elapsed time: ', time.time() - s)
print(best_feature, nested_oos_sharpe, oos_sharpe)  

Elapsed time:  0.001033782958984375
1 -0.10962305159211973 -0.11563006543722415


In [34]:
np.random.seed(42)

n, p = 5000,20

X = np.random.normal(0,1,(n,p))
y = np.random.normal(0,1,n)

s = time.time()
best_feature, nested_oos_sharpe, oos_sharpe = approx_nested_loocv_linear_model_select(X, y)
print('Elapsed time: ', time.time() - s)
print(best_feature, nested_oos_sharpe, oos_sharpe)    

s = time.time()
best_feature, nested_oos_sharpe, oos_sharpe = approx_nested_loocv_linear_model_select_new(X, y)
print('Elapsed time: ', time.time() - s)
print(best_feature, nested_oos_sharpe, oos_sharpe)    





Elapsed time:  0.0030918121337890625
3 -0.3975729072959555 0.002435491055545056
Elapsed time:  0.0
3 -0.3975729072959554 0.0024354910555450834
